In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
batchsize = 400
learningrate = 0.005
epochs = 500 
print_freq = 10

In [3]:
transform = transforms.Compose(
 [transforms.ToTensor(),
 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
tensor_x = torch.from_numpy(np.load('./drive/My Drive/Colab Notebooks/X_train.npy'))
tensor_y = torch.from_numpy(np.load('./drive/My Drive/Colab Notebooks/Y_train.npy'))
tensor_x = torch.reshape(tensor_x, (tensor_x.size(0), tensor_x.size(1), tensor_x.size(2), 1))
print(tensor_x.shape)

torch.Size([10000, 500, 500, 1])


In [4]:
trainset = utils.TensorDataset(tensor_x,tensor_y) 
trainloader = utils.DataLoader(trainset,  batch_size=batchsize)
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(device)

cpu


In [5]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(images.size())
images = images.permute(0, 3, 1, 2)
print(images.size())

torch.Size([400, 500, 500, 1])
torch.Size([400, 1, 500, 500])


In [6]:
def poisson_noise(X_torch):
    X = X_torch.cpu().numpy()
    vals = len(np.unique(X))
    vals = 2 ** np.ceil(np.log2(vals))
    noisy = np.random.poisson(X * vals) / float(vals)
    return noisy

def speckle_noise(X_torch):
    X = X_torch.cpu().numpy()
    X = X.reshape((X.shape[0], X.shape[1])) 
    row,col = X.shape
    s_vs_p = 0.5
    amount = 0.02
    out = np.copy(X)
    num_salt = np.ceil(amount * X.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in X.shape]
    out[coords] = 1
    num_pepper = np.ceil(amount* X.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in X.shape]
    out[coords] = 0
    return out.reshape((out.shape[0], out.shape[1],1))

def flip_2D(X_torch):
    return np.fliplr(X_torch.cpu().numpy())

def augment_batch(inputs, labels):
    x = np.zeros((inputs.size()[0] * 2 ,inputs.size()[1],inputs.size()[2], inputs.size()[3]))
    y = np.zeros((labels.size()[0] * 2))
    for i in range(len(inputs)):
        x[2*i] = inputs[i].cpu().numpy()
        y[2*i] = labels[i].cpu().numpy()
        if (i%3 == 2):
          x[2*i+1] = flip_2D(inputs[i])
          y[2*i+1] = labels[i]
        elif (i%3 == 0):
            x[2*i+1] = poisson_noise(inputs[i])
            y[2*i+1] = labels[i]
        else:
            x[2*i+1] = speckle_noise(inputs[i])
            y[2*i+1] = labels[i]
    return torch.from_numpy(x), torch.from_numpy(y)

In [7]:
class Net(nn.Module):

    def __init__(self, num_classes=2):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 50, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(50, 50, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(50, 100, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(100, 75, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(75, 30, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(1080, 540),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(540, 540),
            nn.ReLU(inplace=True),
            nn.Linear(540, num_classes),
        )

    def forward(self, x):
        x = self.features(x.float())
        x = self.avgpool(x.float())
        x = x.view(x.size(0), 1080)
        x = self.classifier(x.float())
        return x
    
net = Net()
net.to(device)

Net(
  (features): Sequential(
    (0): Conv2d(1, 50, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(50, 50, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(50, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(100, 75, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(75, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=1080, out_features=540, bias=True)
    (2): Re

In [ ]:
loss_func = nn.CrossEntropyLoss()
opt = torch.optim.AdamW(net.parameters(), lr=learningrate)
avg_losses = []
for epoch in range(epochs): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        #inputs, labels = augment_batch(inputs, labels)
        
        inputs = inputs.permute(0, 3, 1, 2)
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()
        outputs = net(inputs)
        loss = loss_func(outputs, labels.long())

        loss.backward()

        opt.step()
        
        running_loss += loss.item()
        if i % print_freq == print_freq - 1:
            avg_loss = running_loss / print_freq
            print('[epoch: {}, i: {:5d}] avg mini-batch loss: {:.3f}'.format(epoch, i, avg_loss))
            avg_losses.append(avg_loss)
            running_loss = 0.0
print('Finished Training.')

[epoch: 0, i:    19] avg mini-batch loss: 292.512
[epoch: 1, i:    19] avg mini-batch loss: 0.649
[epoch: 2, i:    19] avg mini-batch loss: 0.589
[epoch: 3, i:    19] avg mini-batch loss: 0.580
[epoch: 4, i:    19] avg mini-batch loss: 0.581
[epoch: 5, i:    19] avg mini-batch loss: 0.580
[epoch: 6, i:    19] avg mini-batch loss: 0.580
